# Association Rules Mining / Basket Analysis

Basket Analysis (BA) is a powerful technique to mining the assocation between items within a basket, the result could be communicated as **Frequently Bought Together** items, that is used to:

- Create the bundle of items
- Recommend other items to add to basket (as "Frequently Bought Togther")

BA is recommendation algorithm, but not personalization, as it is item-based, and not personalize the recommend by user.

**The interesting of this hands-on**:

- Instead of using any available public dataset, we will generate the synthetic data, by that:
    - We understand deeply the data structure used for this model
    - We can validate if the output make sense or not
- Instead of using the existing Python package (such as: `mlxtend.frequent_patterns`), we will code the logic of Apriori Algorithm by our own, _it is the best way to understand the model in deeper level_



In [ ]:
import os
import sys
from typing import Dict
from numpy.random import choice, randint
from itertools import groupby
import pandas as pd
import pytest
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
import string
from graphviz import Digraph

# Create basket synthetic data

Structure of data input for basket analysis:

- basket_1: apple, egg, milk
- basket_2: carrot, milk
- basket_3: apple, egg, carrot
- basket_4: apple, egg
- basket_5: apple, carrot

## def create_basket

In [619]:
def create_basket(list_item: list, init_freq: list, n_basket: int) -> Dict:
    """Create a basket of items.
    Args:
        list_item (list): list of items ['beer', 'egg', 'cheese', 'bread']
        init_freq (list): initial frequency of items [0.2, 0.3, 0.1, 0.4]
    Returns:
        Dict with key as basket_id (range of int from 0 to n_basket), value is first item of basket
        Example: {1: 'apple', 2: 'egg', 3: 'orange'}
    """
    baskets = {}
    for basket_id in range(n_basket):
        # TODO: Fill in __
        # Hint: Google numpy.random.choice
#         baskets[basket_id] = choice('FILLME_1', 'FILLME_2')
        baskets[basket_id] = choice(list_item, p=init_freq)
    return baskets

In [ ]:
# Test function
baskets_dict_test = create_basket(list_item=['beer', 'egg', 'cheese'], init_freq=[0.5, 0.5, 0.], n_basket=5)
## Test 1
for item in baskets_dict_test.values():
    assert item in ['beer', 'egg', 'cheese']
## Test 2
assert len(baskets_dict_test.keys()) == 5
print('PASS !!!!!')

## def generate_next_items_by_associated_rule

In [ ]:
# associated_rules = {'bread': {'egg': 0.3, 'cheese': 0.2},
#                     'egg': {'apple': 0.2},
#                     'cheese': {'egg': 0.3},
#                     'beer': {'cheese': 0.2, 'wine': 0.5},
#                     'apple': {'orange': 0.2}
#                    }

# synthetic_item_list = ['beer', 'egg', 'cheese', 'bread', 'wine', 'apple', 'orange']
def generate_next_items_by_associated_rule(item, associated_rules, synthetic_item_list):
    """Pre-defined logic of association
    Args:
        item: seeding item
        associated_rules: we pre-define the rules, to see if the model succesfully to detect it
        synthetic_item_list: full list of item
    Returns:
        next item
    """
    try:
        # (1) By the item inputs, extract the associated_items from the rules
        associated_items = list(associated_rules['FILLME_1'].keys())
        # (2) By the item inputs, extract the freqs of those associated_items from the rules
        associated_items_freqs = list(associated_rules['FILLME_2'].values())
        # (3) Take other items not the list of associated_items, and item itself
        other_items = [i for i in 'FILLME_3' if i not in [item, *associated_items]]
        # (4) The prob of other items could be equally random: (1 - sum(associate_prob)) / each itme
        other_freq = 'FILLME_4'
        other_freqs = np.repeat(other_freq, len(other_items)-1)
        next_item = choice([*associated_items, *other_items], 
                           p=[*associated_items_freqs, 
                              *other_freqs, 
                              1. - np.sum(associated_items_freqs) - np.sum(other_freqs)])
    except:
        other_items = [i for i in synthetic_item_list if i != item]
        next_item = choice(other_items)
    return next_item

In [625]:
print('EXAMPLE')
for item in synthetic_item_list:
    next_item = generate_associated_items_by_rule(item, associated_rules, synthetic_item_list)
    print(f'{item} ==> {next_item}')

EXAMPLE
beer ==> wine
egg ==> apple
cheese ==> apple
bread ==> cheese
wine ==> orange
apple ==> beer
orange ==> beer


## Parameters

In [620]:
# Parameters
synthetic_item_list = ['beer', 'egg', 'cheese', 'bread', 'wine', 'apple', 'orange']
synthetic_init_freq = [0.2, 0.05, 0.1, 0.3, 0.05, 0.2, 0.1]

In [622]:
associated_rules = {'bread': {'egg': 0.3, 'cheese': 0.2},
                    'egg': {'apple': 0.2},
                    'cheese': {'egg': 0.3},
                    'beer': {'cheese': 0.2, 'wine': 0.5},
                    'apple': {'orange': 0.2}
                   }

## Initialize basket

In [621]:
# Initialize the basket_dict with 1st item from synthetic_item_list, synthetic_init_freq, with 1000 baskets
baskets_dict = create_basket('FILLME1')

In [631]:
# Create synthetic
basket_df = pd.DataFrame({
    'basket_id': baskets_dict.keys(),
    'items_list': [[item] for item in baskets_dict.values()],
    'size_basket': randint(low=2, high=4, size=1000, dtype=int)
})

# (1) Loop over the maximum size of basket, and add new_item into items_lists based on the last item in item_list
for i in range(basket_df.size_basket.max()):
    basket_df['items_list'] = (basket_df.items_list
                               .apply(
                                   lambda x: [*x, 
                                              # Hint: FILLME_1 is a function
                                              'FILLME_1'(x[-1], 'FILLME_2', 'FILLME_3')
                                             ]
                                     )
                              )

basket_df['items_list'] = basket_df.items_list.apply(lambda x: list(dict.fromkeys(x))) # dedup
# (2) Trim the basket so the num_items = size_baskets
basket_df['items_list'] = basket_df.apply(lambda r: r.items_list[:'FILLME_4'], axis=1)

basket_df.drop(columns='size_basket', inplace=True)

In [632]:
# The data input for BA will look like this
basket_df.head()

,basket_id,items_list
0,0,"[bread, beer, wine]"
1,1,"[cheese, apple, wine]"
2,2,"[orange, cheese]"
3,3,"[orange, wine, egg]"
4,4,"[bread, egg]"


# Apriori Algorithms

## Step 1: Item_support

```
order 1: apple, egg, milk  
order 2: carrot, milk  
order 3: apple, egg, carrot
order 4: apple, egg
order 5: apple, carrot
```

Count how frequent a single items appear in a basket:

```
item set      occurrence count    
{apple}              4   
{egg}                3   
{milk}               2   
{carrot}             2   
```

> We can set some min_occurrence to remove cold-item, for example: `min_occur_thres = 2`, we will remove `{milk}`, and `{carrot}`



In [630]:
df_item_line = (basket_df[['basket_id', 'items_list', 'size_basket']]
                .explode('items_list')
                .rename(columns={'items_list': 'item_name'})
               )
df_item_support = df_item_line.'FILME_1'.'FILLME_2'.nunique().rename('occurence').reset_index() 
df_item_support['item_support'] = df_item_support.occurence / df_item_support.occurence.sum()
df_item_support

,item_name,occurence,item_support
0,apple,340,0.137152
1,beer,372,0.150061
2,bread,457,0.184349
3,cheese,331,0.133522
4,egg,332,0.133925
5,orange,299,0.120613
6,wine,348,0.140379


## Step 2: Create pair of items

1. We need to remove the item not satisfied `min_occur_thres=2`, in our synthetic case, there is no removed item, but in 5-basket example, we have `{apple}` and `{egg}`
2. Calculate the occurence of pair:
```
item set           occurence count  
{apple, egg}             3  
```

In [634]:
# Create all possible pair from the items_list
basket_df['item_pairs'] = basket_df.items_list.apply(lambda x: list('FILLME')) # Hints: itertools.combinations
basket_df.head()

,basket_id,items_list,item_pairs
0,0,"[bread, beer, wine]","[(bread, beer), (bread, wine), (beer, wine)]"
1,1,"[cheese, apple, wine]","[(cheese, apple), (cheese, wine), (apple, wine)]"
2,2,"[orange, cheese]","[(orange, cheese)]"
3,3,"[orange, wine, egg]","[(orange, wine), (orange, egg), (wine, egg)]"
4,4,"[bread, egg]","[(bread, egg)]"


In [636]:
df_item_pairs = basket_df[['basket_id', 'item_pairs']].explode('item_pairs')
df_item_pairs = # FILLME
df_item_pairs.head()

,item_pairs,occurence_AB
0,"(apple, beer)",55
1,"(apple, bread)",53
2,"(apple, cheese)",61
3,"(apple, egg)",52
4,"(apple, orange)",52


## Step 3: Support_AB

The percentage of order that contains the item_pairs: 

```
support{apple,egg} = 3/5 or 60%
```

Later, **minimum support threshold** will need to be set, commonly based on the domain knowlegde. For grocery dataset, for example, it could me many distinct items, so the pair will take only a small fraction of combinations in an basket/order (as low as 0.01% could be accepted)

In [ ]:
# support{apple,egg} = 3/5 or 60%
df_item_pairs['support_AB'] = df_item_pairs.occurence_AB / df_item_pairs.occurence_AB.sum()
df_item_pairs.sort_values(by='support_AB', ascending=False).head()

## Step 4: Confidence

Given two items, A and B, confidence measures the percentage of times that item B is purchased, given that item A was purchased. This is expressed as:

```
confidence{A->B} = support{A,B} / support{A}   
```

```
confidence{B->A} = support{A,B} / support{B}    
```

Confidence values range from 0 to 1, where 0 indicates that B is never purchased when A is purchased, and 1 indicates that B is always purchased whenever A is purchased. Note that the confidence measure is directional. This means that we can also compute the percentage of times that item A is purchased, given that item B was purchased.

First, we need to merge support of item A and B from `df_item_support`

In [ ]:
# Take itemA as the first item in item_pairs
# Take itemB as the second item in item_pairs
df_item_pairs['itemA'] = df_item_pairs.item_pairs.apply(lambda x: 'FILLME_1')
df_item_pairs['itemB'] = df_item_pairs.item_pairs.apply(lambda x: 'FILLME_2')

# Merge item_support of item A from df_item_support
# Hint: rename item_name to itemA, item_support to supportA, similar to B
df_item_pairs = pd.merge(
    df_item_pairs, 
    'FILLME_3'
)
df_item_pairs = pd.merge(
    df_item_pairs, 
    'FILLME_4'
)

Second, calculate `confidence_AB` and `confidence_BA`

In [ ]:
# confidence{A->B} = support{A,B} / support{A}   
df_item_pairs['confidence_AB'] = 'FILLME_1'
df_item_pairs['confidence_BA'] = 'FILLME_2'
df_item_pairs.sort_values(by='support_AB', ascending=False).head()

## Step 5: Lift

Given two items, A and B, lift indicates whether there is a relationship between A and B, or whether the two items are occuring together in the same orders simply by chance (ie: at random)

```
lift{A,B} = lift{B,A} = support{A,B} / (support{A} * support{B})
```

For examople, 
```
  lift{apple,egg} = lift{egg,apple} = support{apple,egg} / (support{apple} * support{egg})
                  = (3/5) / (4/5 * 3/5) 
                  = 1.25  
```
> If apple occurred in 80% of the orders and egg occurred in 60% of the orders, then if there was no relationship between them, we would expect both of them to show up together in the same order 48% of the time (ie: 80% * 60%). The numerator, on the other hand, represents how often apple and egg actually appear together in the same order. In this example, that is 60% of the time. Taking the numerator and dividing it by the denominator, we get to how many more times apple and egg actually appear in the same order, compared to if there was no relationship between them (ie: that they are occurring together simply at random).

Intepretation
```
 * lift = 1 implies no relationship between A and B. 
   (ie: A and B occur together only by chance)

 * lift > 1 implies that there is a positive relationship between A and B.
   (ie:  A and B occur together more often than random)

 * lift < 1 implies that there is a negative relationship between A and B.
   (ie:  A and B occur together less often than random)
```

In [ ]:
# lift{A,B} = lift{B,A} = support{A,B} / (support{A} * support{B}) 
df_item_pairs['lift'] = df_item_pairs.support_AB / (df_item_pairs.supportA * df_item_pairs.supportB)
df_item_pairs.sort_values(by='support_AB', ascending=False).head()

# Deep-dive the results

Now, let's check the distribution of support, confidence, and lift

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(13, 5))

xs = ['lift', 'support_AB', 'confidence_AB']
colors = ['skyblue', 'olive', 'gold']

for i in range(3):
    sns.histplot(data=df_item_pairs, x=xs[i], kde=True, color=colors[i], ax=axs[i])

## min_support, num_rules, confidence

When min_support is higher, we have less numbers of rules (consider with confidence, there are not much left). Let's check on the trade-off between min_support, num_rules, and consideration with confidence level

In [ ]:
def plot_numrules_confidence_lvl(df_item_pairs, min_support, ax):
    rules_df = df_item_pairs[df_item_pairs.support_AB >= min_support]
    confidence_levels = np.arange(0, 1, 0.1)
    num_rules = []
    for thres_lvl in confidence_levels:
        rule_cnt = rules_df[rules_df.confidence_AB > thres_lvl].shape[0]
        num_rules.append(rule_cnt)
    
    df_confidence_rule_cnt = pd.DataFrame({
        'confidence_level': confidence_levels,
        'num_rules': num_rules
    }).set_index('confidence_level')
    
    ax.plot(confidence_levels, num_rules, 'o-', color='black')
    ax.set_title(f'Cut-off: support_AB >= {min_support}')
    ax.set_xlabel('Confidence level')
    return ax

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(13, 5))
min_support = [0.01, 0.03, 0.05]
for i in range(3):
    plot_numrules_confidence_lvl(df_item_pairs, min_support=min_support[i], ax=axs[i])
    if i == 0:
        axs[i].set_ylabel('Number of rules')

## Filter min_support & min_confidence

In [ ]:
df_item_pairs_flr_support = df_item_pairs[df_item_pairs.support_AB >= 0.01]
df_item_pairs_flr_support_confi = df_item_pairs_flr_support[df_item_pairs_flr_support.confidence_AB >= 0.2]
print(df_item_pairs.shape)
print(df_item_pairs_flr_support.shape)
print(df_item_pairs_flr_support_confi.shape)

## Sense-check: lift = 1, lift >> 1

In [ ]:
# lift = 1 implies no relationship between A and B. 
# (ie: A and B occur together only by chance)
df_item_pairs[df_item_pairs.lift.between(0.9, 1.1)].head()

In [ ]:
df_item_pairs_flr_support_confi[df_item_pairs_flr_support_confi.lift.between(0.9, 1.1)].shape

In [ ]:
# lift > 1 implies that there is a positive relationship between A and B.
# (ie:  A and B occur together more often than random)
df_item_pairs_flr_support_confi.sort_values(by='lift', ascending=False).head()

In [ ]:
df_item_pairs_flr_support_confi.sort_values(by='lift', ascending=False).tail()

## Heatmap of lift x items-pairs

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(13, 5))

xs = ['lift', 'support_AB', 'confidence_AB']
colors = ['skyblue', 'olive', 'gold']

tit1 = 'without filter'
tit2 = 'min-support>=0.01'
tit3 = tit2 + ' ' + 'confi-lvl>=0.2'

sns.histplot(data=df_item_pairs, x='lift', kde=True, color='gold', ax=axs[0]).set(title=tit1)
sns.histplot(data=df_item_pairs_flr_support, x='lift', kde=True, color='olive', ax=axs[1]).set(title=tit2)
sns.histplot(data=df_item_pairs_flr_support_confi, x='lift', kde=True, color='skyblue', ax=axs[2]).set(title=tit3);

In [ ]:
df_apriori = df_item_pairs_flr_support_confi.sort_values(by='lift', ascending=False)
df_apriori

In [ ]:
df_heatmap = (df_apriori
              .groupby(['itemA', 'itemB']).lift.mean()
              .reset_index()
              .pivot(index='itemA', columns='itemB', values='lift')
             )
plt.figure(figsize=(7, 5))
sns.heatmap(df_heatmap, annot=True, fmt='.1f', cmap=sns.cubehelix_palette(as_cmap=True))
plt.title('Apriori output: Lift by item pair');

## Further: Filter by lift

In [ ]:
df_apriori = df_apriori[df_apriori.lift >= 1.3]
df_apriori

In [ ]:
df_heatmap = (df_apriori
              .groupby(['itemA', 'itemB']).lift.mean()
              .reset_index()
              .pivot(index='itemA', columns='itemB', values='lift')
             )
plt.figure(figsize=(7, 5))
sns.heatmap(df_heatmap, annot=True, fmt='.1f', cmap=sns.cubehelix_palette(as_cmap=True))
plt.title('Apriori output: Lift by item pair');

## Intro to graph

In [ ]:
# Create nodes_df
associated_items_ls = pd.concat([df_apriori.itemA, df_apriori.itemB]).unique()
nodes_df = df_item_support[df_item_support.item_name.isin(associated_items_ls)]

alphanum_nodeid = list(string.ascii_lowercase) + [str(i) for i in range(9)]
nodes_df['node_id'] = [alphanum_nodeid[i] for i in range(len(nodes_df))]
nodes_df

In [ ]:
# Add node_id to df_apriori
nodes_id_dict = {r.item_name: r.node_id for _, r in nodes_df.iterrows()}
df_apriori['from_node_id'] = df_apriori.itemA.apply(lambda x: nodes_id_dict[x])
df_apriori['to_node_id'] = df_apriori.itemB.apply(lambda x: nodes_id_dict[x])
df_apriori['edge'] = df_apriori.from_node_id + df_apriori.to_node_id
df_apriori

In [ ]:
scaled_size = 4
gra = Digraph(edge_attr={'color':'lightgray'},
              node_attr={'shape': 'circle', 'fixedsize':'true', 'style': 'filled', 'fontsize': '10'},
              graph_attr={'beautify':'true', 'margin': '0.5'},
              format='pdf'
             )

for _, r in nodes_df.iterrows():
    gra.node(r.node_id, r.item_name.lower(), **{'width':str(r.item_support * scaled_size)})

gra.edges(df_apriori.edge.tolist())
gra